In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
#https://stackoverflow.com/questions/38208700/matplotlib-plot-lines-with-colors-through-colormap
from scipy.optimize import curve_fit
#https://stackoverflow.com/questions/10208814/colormap-for-errorbars-in-x-y-scatter-plot-using-matplotlib
import matplotlib
import matplotlib.cm as cm

In [ ]:
def plot_xy(x_val, y_val, x_error=float("nan"), y_error=float("nan"), err_amp=1,fig = None, ax=None,lines=None,labels=None,titel=None,x_label="x-Axis",y_label="y-Axis", color="blue"):
#x_val, y_val, x_error, y_error,fig,ax,lines,labels,titel,x_label,y_label, color
    assert len(x_val) == len(y_val)
    if type(x_error) == float:
        x_error = np.ones_like(x_val) * x_error
    if type(y_error) == float:
        y_error = np.ones_like(y_val) * y_error

    if ax is None:
        #figure
        fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,6)) 
    if lines is None:
        lines=list()
    if labels is None:
        labels=list()
    witherr = False
    noerr = False
    #loop over each data point to plot
    for x, y, xe, ye in zip(x_val, y_val, x_error, y_error):
        if (x != x) or (y != y):
            continue
        if (xe == xe) and (ye == ye):
            witherr = True
            ax.plot(x, y, 'x', color=color)
            ax.errorbar(x, y, xerr=xe*err_amp, yerr=ye*err_amp, lw=1, capsize=1, color=color)
        elif (xe == xe):
            witherr = True
            ax.plot(x, y, 'x', color=color)
            ax.errorbar(x, y, xerr=xe*err_amp, lw=1, capsize=1, color=color)
        elif (ye == ye):
            witherr = True
            ax.plot(x, y, 'x', color=color)
            ax.errorbar(x, y, yerr=ye*err_amp, lw=1, capsize=1, color=color)
        else:
            noerr = True
            ax.plot(x, y, 'o', color=color)

    ax.set_xlabel(x_label, fontsize=16)
    ax.set_ylabel(y_label, fontsize=16)
    if not titel is None:
        ax.set_title(titel, fontsize=16)
    #ax.set_facecolor("grey")

    if witherr:
        witherr, = plt.plot([], [], 'x', color=color)
        lines.append(witherr)
        if err_amp != 1:
            labels.append("Simulation data with {}-fold SEM".format(err_amp))
        else:
            labels.append("Simulation data with SEM")
    if noerr:
        noerr, = plt.plot([], [], 'o', color=color)
        lines.append(noerr)
        labels.append("Simulation data without error")        
    ax.legend(lines, labels, loc="best")
    return fig, ax, lines, labels

In [ ]:
def plot_xyz(x_val, y_val, z_val, x_error=float("nan"), y_error=float("nan"), err_amp=1,ax=None,fig=None,titel=None,x_label="x-Axis",y_label="y-Axis",z_label="z-Axis"):
    assert len(x_val) == len(y_val)
    assert len(x_val) == len(z_val)
    if type(x_error) == float:
        x_error = np.ones_like(x_val) * x_error
    if type(y_error) == float:
        y_error = np.ones_like(y_val) * y_error

    if ax is None:
        #figure
        fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,6)) 

    #create a scatter plot
    sc = ax.scatter(x_val,y_val,s=0,c=z_val, cmap='hot')

    #create colorbar according to the scatter plot
    clb = plt.colorbar(sc,label=z_label, cmap='hot')

    #convert time to a color tuple using the colormap used for scatter
    norm = matplotlib.colors.Normalize(vmin=min(z_val), vmax=max(z_val), clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap='hot')
    z_color = np.array([(mapper.to_rgba(v)) for v in z_val])
    
    witherr = False
    noerr = False
    #loop over each data point to plot
    for x, y, xe, ye, color in zip(x_val, y_val, x_error, y_error, z_color):
        if (x != x) or (y != y):
            continue
        if (xe == xe) and (ye == ye):
            witherr = True
            ax.plot(x, y, 'x', color=color)
            ax.errorbar(x, y, xerr=xe*err_amp, yerr=ye*err_amp, lw=1, capsize=1, color=color)
        elif (xe == xe):
            witherr = True
            ax.plot(x, y, 'x', color=color)
            ax.errorbar(x, y, xerr=xe*err_amp, lw=1, capsize=1, color=color)
        elif (ye == ye):
            witherr = True
            ax.plot(x, y, 'x', color=color)
            ax.errorbar(x, y, yerr=ye*err_amp, lw=1, capsize=1, color=color)
        else:
            noerr = True
            ax.plot(x, y, 'o', color=color)

    ax.set_xlabel(x_label, fontsize=16)
    ax.set_ylabel(y_label, fontsize=16)
    if not titel is None:
        ax.set_title(titel, fontsize=16)
    ax.set_facecolor("grey")
    
    lines = list()
    labels = list()
    if witherr:
        witherr, = ax.plot([], [], 'x', color="blue")
        lines.append(witherr)
        if err_amp != 1:
            labels.append("Simulation data with {}-fold SEM".format(err_amp))
        else:
            labels.append("Simulation data with SEM")
    if noerr:
        noerr, = ax.plot([], [], 'o', color="blue")
        lines.append(noerr)
        labels.append("Simulation data without error")        
    ax.legend(lines, labels)
    return fig, ax

In [ ]:
#tanh
def fkt_tanh(x,a,b,c,d):
    return a * np.tanh(b*(x-c)) + d

def fkt_const(x,c):
    return c
fkt_const = np.vectorize(fkt_const)

def temperature_Phi_ising(x):
    return 1/(2*np.arcsinh((1-(2*x-1)**8)**(-1/4)))

if False:
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,6))
    X = np.linspace(-10,10,1000)
    a=-0.5
    b=1
    c=3
    d=0.5
    ax.plot(X, fkt_tanh(X,a,b,c,d), label="tanh")
if False:    
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,6))
    T = np.linspace(1e-1,2*np.arcsinh(1)**-1,1000)
    def m_t(t):
        return (1-np.sinh(2/t)**-4)**(1/8)

    ax.plot(T,m_t(T), label="m(T)")
if False:
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,6)) 
    X = np.linspace(0,1,1000)
    X[0] = 1e-9
    X[-1] = 1 - 1e-9
    ax.plot(X,temperature_Phi_ising(X), label="T(Phi)")
T_cIsing = temperature_Phi_ising(0.5)
print(T_cIsing)

if True:
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,6)) 
    X = np.linspace(0.0,0.6,1000)
    
    def h1(T):
        return 0.35
    h1 = np.vectorize(h1)
    ax.plot(X,h1(X), label="h = 0.6")
    
    def h2(T):
        return 0.45 - T * 0.4
    h2 = np.vectorize(h2)
    ax.plot(X,h2(X), label="h = 0.6 - T * 0.4")
    
    def h3(T,T_c=T_cIsing):
        return 0.35 - T * 6 * (T - T_c)
    h3 = np.vectorize(h3)
    ax.plot(X,h3(X), label="h = 0.6 - T * 12 * (T - T_c)")
    
    
    ax.legend(loc="best")
    ax.set_title("h(T)")

In [ ]:
def h3(T,T_c=T_cIsing):
    return 0.4 - T * 6 * (T - T_c)


def Phi_MFT3(x,t):
    T_cMFT = 0.25
    return (h3(t,T_c=T_cMFT)+t*np.log(x/(1-x)))/x

fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (4,6))

T = np.array([0.17,0.2,0.23])
X = np.linspace(0.0001,0.999,100)

for t in T:
    ax.plot(X,Phi_MFT3(X,t))
    
ax.set_xlim(0,1)
ax.set_ylim(0.6,1)

In [ ]:
def x_from_phi(phi, err_phi):
        phi_ges = phi[0] + phi[1]
        frac = phi[1] / phi_ges
        err_phi_ges = np.sqrt(err_phi[0]**2 + err_phi[1]**2)
        err_frac = np.sqrt((phi[0]*err_phi[1])**2 + (phi[1]*err_phi[0])**2)/phi_ges**2
        return phi_ges, err_phi_ges, frac, err_frac

In [ ]:
#multiple simulations
#https://stackoverflow.com/questions/38208700/matplotlib-plot-lines-with-colors-through-colormap
def multilineOneX(X, ys, c, ax, **kwargs):
        """Plot lines with different colorings

        Parameters
        ----------
        xs : iterable container of x coordinates
        ys : iterable container of y coordinates
        c : iterable container of numbers mapped to colormap
        ax (optional): Axes to plot on.
        kwargs (optional): passed to LineCollection

        Notes:
            len(xs) == len(ys) == len(c) is the number of line segments
            len(xs[i]) == len(ys[i]) is the number of points for each line (indexed by i)

        Returns
        -------
        lc : LineCollection instance.
        """

        # create LineCollection
        segments = [np.column_stack([X, y]) for y in ys]
        lc = LineCollection(segments, **kwargs)

        # set coloring of line segments
        #    Note: I get an error if I pass c as a list here... not sure why.
        lc.set_array(np.asarray(c))

        # add lines to axes and rescale 
        #    Note: adding a collection doesn't autoscalee xlim/ylim
        ax.add_collection(lc)
        ax.autoscale()
        return lc
    
def multiline(xs, ys, c, ax=None, **kwargs):
    """Plot lines with different colorings

    Parameters
    ----------
    xs : iterable container of x coordinates
    ys : iterable container of y coordinates
    c : iterable container of numbers mapped to colormap
    ax (optional): Axes to plot on.
    kwargs (optional): passed to LineCollection

    Notes:
        len(xs) == len(ys) == len(c) is the number of line segments
        len(xs[i]) == len(ys[i]) is the number of points for each line (indexed by i)

    Returns
    -------
    lc : LineCollection instance.
    """

    # find axes
    ax = plt.gca() if ax is None else ax

    # create LineCollection
    segments = [np.column_stack([x, y]) for x, y in zip(xs, ys)]
    lc = LineCollection(segments, **kwargs)

    # set coloring of line segments
    #    Note: I get an error if I pass c as a list here... not sure why.
    lc.set_array(np.asarray(c))

    # add lines to axes and rescale 
    #    Note: adding a collection doesn't autoscalee xlim/ylim
    ax.add_collection(lc)
    ax.autoscale()
    return lc
    
def plotMultipleDensty(xData, filename="dateinamen vergessen"):
    #sort data
    xdim = int(xData[0,0])#assuming having always the same lattice length
    n_sim = np.shape(xData)[0]
    T = xData[:,1]
    coverage = xData[:,2]
    meanPhi1 = xData[:,3:xdim+3]
    meanPhi2 = xData[:,xdim+3:]

    assert np.size(meanPhi2) == np.size(meanPhi1)
    meanPhi12 = np.array([meanPhi1, meanPhi2])
    
    #Plot
    X = np.linspace(0,xdim-1,xdim)
    lc = list()
    axcb = list()
    fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (14,10))
    
    for i in range(2):
        lc.append(multilineOneX(X, meanPhi12[i], T, ax[i], cmap='hot'))
        axcb.append(fig.colorbar(lc[i], ax=ax[i]))
        axcb[i].set_label("Temperature T")
        ax[i].set_xlabel("Position along the long lattice direction")
        ax[i].set_ylabel("Average occupation Phi_{}=N_{}/N_total".format(i+1,i+1))
        ax[i].set_facecolor("grey")
    
    ax[0].set_title("Density distribution of lying Molecules", fontsize=16)
    ax[1].set_title("Density distribution of standing Molecules", fontsize=16)
    fig.savefig(filename)
    
    
#plotMultipleDensty(xData, "density_distribution")

In [ ]:
def fitMultipleDenstyTanh(xData, filename="dateinamen vergessen", T_c = T_cIsing):
    "in dieser Funktion gilt X = <N_i>/ydim, also eintlich Phi_i"
    #sort data
    xdim = int(xData[0,0])#assuming having always the same lattice length
    n_sim = np.shape(xData)[0]
    T = xData[:,1]
    coverage = xData[:,2]
    meanX1 = xData[:,3:xdim+3]
    meanX2 = xData[:,xdim+3:]

    assert np.size(meanX2) == np.size(meanX1)
    meanX = np.array([meanX1, meanX2])
    
    X = np.linspace(0,xdim-1,xdim)
    
    #fit
    paras_left = [[],[]]#[[[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]],...],[[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]],...]]
    paras_right = [[],[]]#i x sim x [para,err] x 4
    for i in range(2):
        for sim in range(n_sim):
            #print("i = {} sim = {} T = {}".format(i,sim,T[sim]))
            #left
            if T[sim] < T_cIsing:
                try:
                    popt, pcov = curve_fit(fkt_tanh,X[0:xdim//2],meanX[i,sim,0:xdim//2],[i/2.0,1.0,xdim/4.0,0.5])
                    perr = np.sqrt(np.diag(pcov))
                    #manually correct for not detecting flat line
                    if (np.sum(meanX[i,sim,0:xdim//2]) == 0):
                        popt[0] = 0
                        popt[3] = 0
                    paras_left[i].append([popt,perr])
                except:
                    #paras_left[i].append([[0,0,0,-0.01*(1+sim)],[float("nan"),float("nan"),float("nan"),float("nan")]])
                    paras_left[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
                #right
                try:
                    popt, pcov = curve_fit(fkt_tanh,X[xdim//2:],meanX[i,sim,xdim//2:],[i/2.0,-1.0,3*xdim/4.0,0.5])
                    perr = np.sqrt(np.diag(pcov))
                    #manually correct for not detecting flat line
                    if (np.sum(meanX[i,sim,xdim//2:]) == 0):
                        popt[0] = 0
                        popt[3] = 0
                    paras_right[i].append([popt,perr])
                except:
                    #paras_right[i].append([[0,0,0,-0.01*(1+sim)],[float("nan"),float("nan"),float("nan"),float("nan")]])
                    paras_right[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
            else:
                paras_left[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
                paras_right[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
                
    paras_left = np.array(paras_left) 
    paras_right = np.array(paras_right)
    #print(paras_left)
    #print(paras_right)
    
    #Plot
    lc = list()
    axcb = list()
    fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (12,8))
    
    for i in range(2):
        lc.append(multilineOneX(X, meanX[i], T, ax[i], cmap='hot', linestyle = "solid"))
        
        Y=np.array([fkt_tanh(X[0:xdim//2],*paras_left[i,sim,0]) for sim in range(n_sim)])
        lc.append(multilineOneX(X[0:xdim//2], Y, T, ax[i], cmap='hot', linestyle = "dotted"))
        
        Y=np.array([fkt_tanh(X[xdim//2:],*paras_right[i,sim,0]) for sim in range(n_sim)])
        lc.append(multilineOneX(X[xdim//2:], Y, T, ax[i], cmap='hot', linestyle = "dotted"))
        
        axcb.append(fig.colorbar(lc[i], ax=ax[i]))
        axcb[i].set_label(r'Temperature $T$', fontsize=16)
        ax[i].set_facecolor("grey")
    
    #ax[0].set_title("Coverage of lying molecules", fontsize=16)
    ax[0].set_ylabel('Coverage of lying\n'+'molecules $\Phi_1$', fontsize=16)
    #ax[1].set_title("Coverage of standing molecules", fontsize=16)
    ax[1].set_ylabel('Coverage of standing\n'+'molecules $\Phi_2$', fontsize=16)
    ax[1].set_xlabel(r'Position $x$', fontsize=16)
    for i in range(2):
        lines = list()
        labels = list()
        line, = ax[i].plot([],[], color="blue", linestyle = "solid")
        lines.append(line)
        labels.append("Simulation data")
        line, = ax[i].plot([],[], color="blue", linestyle = "dotted")
        lines.append(line)
        labels.append("Fitted functions")
        ax[i].legend(lines, labels,loc="best")
    fig.savefig(filename)

    #side x i x sim x [para,err] x 4
    return np.array([paras_left, paras_right])
    
#paras = fitMultipleDenstyTanh(xData, "density_fit_plot")

In [ ]:
def fitMultipleDenstyTanhJustStanding(xData, filename="dateinamen vergessen", T_c = T_cIsing):
    "in dieser Funktion gilt X = <N_i>/ydim, also eintlich Phi_i"
    #sort data
    xdim = int(xData[0,0])#assuming having always the same lattice length
    n_sim = np.shape(xData)[0]
    T = xData[:,1]
    coverage = xData[:,2]
    meanX1 = xData[:,3:xdim+3]
    meanX2 = xData[:,xdim+3:]

    assert np.size(meanX2) == np.size(meanX1)
    meanX = np.array([meanX1, meanX2])
    
    X = np.linspace(0,xdim-1,xdim)
    
    #fit
    paras_left = [[],[]]#[[[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]],...],[[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]],...]]
    paras_right = [[],[]]#i x sim x [para,err] x 4
    for i in range(2):
        for sim in range(n_sim):
            #print("i = {} sim = {} T = {}".format(i,sim,T[sim]))
            #left
            if T[sim] < T_cIsing:
                try:
                    popt, pcov = curve_fit(fkt_tanh,X[0:xdim//2],meanX[i,sim,0:xdim//2],[i/2.0,1.0,xdim/4.0,0.5])
                    perr = np.sqrt(np.diag(pcov))
                    #manually correct for not detecting flat line
                    if (np.sum(meanX[i,sim,0:xdim//2]) == 0):
                        popt[0] = 0
                        popt[3] = 0
                    paras_left[i].append([popt,perr])
                except:
                    #paras_left[i].append([[0,0,0,-0.01*(1+sim)],[float("nan"),float("nan"),float("nan"),float("nan")]])
                    paras_left[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
                #right
                try:
                    popt, pcov = curve_fit(fkt_tanh,X[xdim//2:],meanX[i,sim,xdim//2:],[i/2.0,-1.0,3*xdim/4.0,0.5])
                    perr = np.sqrt(np.diag(pcov))
                    #manually correct for not detecting flat line
                    if (np.sum(meanX[i,sim,xdim//2:]) == 0):
                        popt[0] = 0
                        popt[3] = 0
                    paras_right[i].append([popt,perr])
                except:
                    #paras_right[i].append([[0,0,0,-0.01*(1+sim)],[float("nan"),float("nan"),float("nan"),float("nan")]])
                    paras_right[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
            else:
                paras_left[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
                paras_right[i].append([[float("nan"),float("nan"),float("nan"),float("nan")],[float("nan"),float("nan"),float("nan"),float("nan")]])
                
    paras_left = np.array(paras_left) 
    paras_right = np.array(paras_right)
    #print(paras_left)
    #print(paras_right)
    
    #Plot
    lc = list()
    axcb = list()
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (12,5))
    
    for i in range(1):
        lc.append(multilineOneX(X, meanX[1], T, ax, cmap='hot', linestyle = "solid"))
        
        Y=np.array([fkt_tanh(X[0:xdim//2],*paras_left[1,sim,0]) for sim in range(n_sim)])
        lc.append(multilineOneX(X[0:xdim//2], Y, T, ax, cmap='hot', linestyle = "dotted"))
        
        Y=np.array([fkt_tanh(X[xdim//2:],*paras_right[1,sim,0]) for sim in range(n_sim)])
        lc.append(multilineOneX(X[xdim//2:], Y, T, ax, cmap='hot', linestyle = "dotted"))
        
        axcb.append(fig.colorbar(lc[0], ax=ax))
        axcb[0].set_label(r'Temperature $T$', fontsize=16)
        ax.set_xlabel(r'Position $x$', fontsize=16)
        ax.set_facecolor("grey")
    
    
    ax.set_ylabel('Coverage of standing\n'+' molecules $\Phi_2$', fontsize=16)
    lines = list()
    labels = list()
    line, = ax.plot([],[], color="blue", linestyle = "solid")
    lines.append(line)
    labels.append("Simulation data")
    line, = ax.plot([],[], color="blue", linestyle = "dotted")
    lines.append(line)
    labels.append("Fitted functions")
    ax.legend(lines, labels,loc="best")
    fig.savefig(filename)

    #side x i x sim x [para,err] x 4
    return np.array([paras_left, paras_right])

In [ ]:
def single_set_paras_to_Phi(xdim, paras):
    #[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]]
    
    #don't need fitting errors, because erros are calculated differently
    just_values = paras[0]
    a,b,c,d = just_values
    
    
    #check if fit was succesfull and if domain boundary is within the lattice
    array_sum = np.sum(paras)
    if (np.isnan(array_sum) or c < 0 or c > xdim):
        x = float("nan")
        return np.array([x, x])    
    
    #side -1 => left
    #side 1 => right
    side = -1
    if c > xdim/2:
        side = 1
    
    signb = np.sign(b)
    
    #centre is the phase around xdim/2
    Phi_outer = d + side * signb * a
    Phi_centre = d - side * signb * a
    
    return np.array([Phi_outer, Phi_centre])
    #trys to return [Phi_outer, Phi_centre]

def try_find_mean(left, right):
    #can be nan
    if np.isnan(left):
        return np.array(right)
    
    if np.isnan(right):
        return np.array(left)
    
    #both values are valid
    return (left + right) / 2    
    
def paras_to_Phi(xdim, paras):
    #[[[[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]],...],[[[a,b,c,d],[sd_a,sd_b,sd_c,sd_d]],...]]]
    #side x i x sim x [para,err] x 4
    n_i = np.shape(paras)[1]
    n_sim = np.shape(paras)[2]
    
    #i x sim x phase
    Phi = np.zeros((n_i,n_sim,2))
    
    for i in range(n_i):  
        for sim in range(n_sim):
            Phi_left = single_set_paras_to_Phi(xdim, paras[0,i,sim])
            Phi_right = single_set_paras_to_Phi(xdim, paras[1,i,sim])
            for phase in range(2):
                Phi[i,sim,phase] = try_find_mean(Phi_left[phase],Phi_right[phase])    
    
    return Phi #i x sim x phase
     
         
             
#Phi_final = paras_to_Phi(xdim, paras)
#i x sim x phase([outer,centre])
#print(Phi_final)

In [ ]:
def makePhiPhysical(PhiFromParas):
    #i x sim x phase
    #filters out unphysical values
    PhiPhysical = np.zeros_like(PhiFromParas)
    for i in range(2):
        for sim in range(np.shape(PhiFromParas)[1]):
            for phase in range(2):
                if 0. <= PhiFromParas[i,sim,phase] <= 1.:
                    PhiPhysical[i,sim,phase] = PhiFromParas[i,sim,phase]
                elif 1. < PhiFromParas[i,sim,phase] < 1.01:
                    PhiPhysical[i,sim,phase] = 1.
                elif -0.01 < PhiFromParas[i,sim,phase] < 0.:
                    PhiPhysical[i,sim,phase] = 0
                else:
                    PhiPhysical[i,sim,phase] = float("nan")
                    
    return PhiPhysical

In [ ]:
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3487226/
def mean_SEM(x):
    #1d array
    if np.shape(x)[0] == 1:
        return np.array([x[0], float("nan")])
    n = np.shape(x)[0]
    mean = np.sum(x) / n
    Var = 0
    for i in range(n):
        Var += (x[i] - mean)**2
    Var /= (n - 1)
    SD = np.sqrt(Var)
    SEM = SD / np.sqrt(n)
    return np.array([mean, SEM])

def meanPhiPerT(PhiPhysical,T,coverage):
    #i x sim x phase
    #sim
    #sim
    n_sim = np.shape(T)[0]
    listT = np.sort(list(set(T)))
    listCov = np.sort(list(set(coverage)))
    
    assert np.shape(listCov)[0] == 1#for now till I think about how to include multiple coverages
    if np.shape(listCov)[0] == 1:
        phi = np.zeros((2,np.shape(listT)[0],2,2))
        cov = list()
        for t_j,j in zip(listT,range(len(listT))):
            for i in range(2):
                for phase in range(2):
                    #print(PhiPhysical[i,:,phase])
                    p = np.array([PhiPhysical[i,sim,phase] for sim in range(n_sim) if (t_j == T[sim])])
                    p = mean_SEM(p)
                    phi[i,j,phase] = p       
                    
        #i x sim x phase x [val,err]
        return phi, listT, np.ones_like(listT)*listCov[0]
    
#phi_mean, T_mean, cov_mean = meanPerT(PhiPhysical,T,coverage)

In [ ]:
def phaseDiagramm_x_Phi_T(Phi,T,coverage,err_amp=1, filename=None,titel=None, ax = None, fig=None, y_label=True):
    #i x sim x phase x [Phi,err]
    assert len(np.shape(Phi)) == 4
    phi = Phi[:,:,:,0]
    err_phi = Phi[:,:,:,1]    
    
    x_val = list()
    y_val = list()
    z_val = list()
    x_err = list()
    y_err = list()
    
    for sim in range(np.shape(Phi)[1]):
        for phase in range(2):
            phi_ges, err_phi_ges, frac, err_frac = x_from_phi(phi[:,sim,phase], err_phi[:,sim,phase])
            
            x_val.append(frac)
            x_err.append(err_frac)
            y_val.append(phi_ges)
            y_err.append(err_phi_ges)
            z_val.append(T[sim])


    x_val = np.array(x_val)
    y_val = np.array(y_val)
    z_val = np.array(z_val)
    x_err = np.array(x_err)
    y_err = np.array(y_err)
    
    if ax is None:
        fig, ax = plot_xyz(x_val, y_val, z_val, x_error=x_err, y_error=y_err,err_amp=err_amp,x_label=r'Fraction of standing molecules $x_2=\frac{\Phi_2}{\Phi}$',y_label=r'Coverage $\Phi = \Phi_1 + \Phi_2$',z_label=r'Temperature $T$')
    else:
        fig, ax = plot_xyz(x_val, y_val, z_val, x_error=x_err, y_error=y_err,err_amp=err_amp,ax=ax,fig=fig,x_label=r'Fraction of standing molecules $x_2=\frac{\Phi_2}{\Phi}$',y_label=r'Coverage $\Phi = \Phi_1 + \Phi_2$',z_label=r'Temperature $T$')
    #ax.set_xlim([-1e-2,1+1e-2])
    if not y_label:
        ax.set_ylabel=""
    if not titel is None:
        ax.set_title(titel, fontsize=16)
    if filename is None:
        fig.savefig("x-Phi-T-diagramm")
    else:
        fig.savefig(filename)
    
#phaseDiagramm_x_Phi_T(phi_mean,T_mean,cov_mean)

In [ ]:
def phaseDiagramm_x_T(Phi,T,cov,err_amp=1, filename=None, ax = None, fig=None):
    #i x sim x phase x [Phi,err]
    assert len(np.shape(Phi)) == 4
    phi = Phi[:,:,:,0]
    err_phi = Phi[:,:,:,1]    
    
    x_val = list()
    y_val = list()
    x_err = list()
    
    for sim in range(np.shape(Phi)[1]):
        for phase in range(2):
            phi_ges, err_phi_ges, frac, err_frac = x_from_phi(phi[:,sim,phase], err_phi[:,sim,phase])
            x_val.append(frac)
            x_err.append(err_frac)
            y_val.append(T[sim])

    x_val = np.array(x_val)
    y_val = np.array(y_val)
    x_err = np.array(x_err)
    
    #Plot
    if ax is None:
        fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,7))
    fig, ax, lines, labels  = plot_xy(x_val, y_val, x_error=x_err,err_amp=err_amp,fig=fig,ax=ax,x_label=r'Fraction $x_2=\frac{\Phi_2}{\Phi}$',y_label=r'Temperature $T$')
    fig.savefig("x-T-diagramm")
    
#phaseDiagramm_x_T(phi_mean,T_mean,cov_mean)

In [ ]:
def phaseDiagramm_T_phi(Phi,T,coverage,err_amp=1, filename=None, ax = None, fig=None):
    #i x sim x phase x [Phi,err]
    assert len(np.shape(Phi)) == 4
    phi = Phi[:,:,:,0]
    err_phi = Phi[:,:,:,1]    
    
    x_val = list()
    y_val = list()
    y_err = list()
    
    for sim in range(np.shape(T)[0]):
        for phase in range(2):
            phi_ges, err_phi_ges, frac, err_frac = x_from_phi(phi[:,sim,phase], err_phi[:,sim,phase])
            x_val.append(T[sim])
            y_val.append(phi_ges)
            y_err.append(err_phi_ges)
                
            

    x_val = np.array(x_val)
    y_val = np.array(y_val)
    y_err = np.array(y_err)
        
    #Plot
    if ax is None:
            fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,7))
    fig, ax, lines, labels  = plot_xy(x_val, y_val, y_error=y_err,err_amp=err_amp,fig=fig,ax=ax,x_label=r'Temperature $T$',y_label=r'Coverage $\Phi = \Phi_1 + \Phi_2$')
    X = np.linspace(0,1,1000)
    X[0] = 1e-9
    X[-1] = 1 - 1e-9
    line, = ax.plot(temperature_Phi_ising(X),X, color="orange")
    lines.append(line)
    labels.append("Ising binodal")
    ax.legend(lines, labels,loc="best")
    if filename is None:
        fig.savefig("Phi_12-T-diagramm")
    else:
        fig.savefig(filename)
    
#phaseDiagramm_T_phi(phi_mean,T_mean,cov_mean)

In [ ]:
def phaseDiagramm_Phi12_T(Phi,T,cov,err_amp=1, filename=None, ax = None, fig=None, titel=None, y_label=True):
    #i x sim x phase x [Phi,err]
    assert len(np.shape(Phi)) == 4
    phi = Phi[:,:,:,0]
    err_phi = Phi[:,:,:,1]    
    
    x_val1 = list()
    x_err1 = list()
    x_val2 = list()
    x_err2 = list()
    y_val = list()
    
    for sim in range(np.shape(T)[0]):
        for phase in range(2):
            y_val.append(T[sim])
            x_val1.append(phi[0,sim,phase])
            x_err1.append(err_phi[0,sim,phase])
            x_val2.append(phi[1,sim,phase])
            x_err2.append(err_phi[1,sim,phase])
                
    x_val1 = np.array(x_val1)
    x_err1 = np.array(x_err1)
    x_val2 = np.array(x_val2)
    x_err2 = np.array(x_err2)
    y_val = np.array(y_val)
        
    #Plot
    if ax is None:
        fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,7))
    if y_label:
        fig, ax, lines, labels  = plot_xy(x_val1, y_val, x_error=x_err1,err_amp=err_amp,fig=fig,ax=ax,y_label=r'Temperature $T$',x_label=r'Coverage per configuration $\Phi_{1/2}$',color="red")
        fig, ax, lines, labels  = plot_xy(x_val2, y_val, x_error=x_err2,err_amp=err_amp,fig=fig,ax=ax,lines=lines,labels=labels,y_label=r'Temperature $T$',x_label=r'Coverage per configuration $\Phi_{1/2}$',color="black")
    else:
        fig, ax, lines, labels  = plot_xy(x_val1, y_val, x_error=x_err1,err_amp=err_amp,fig=fig,ax=ax,y_label='',x_label=r'Coverage per configuration $\Phi_{1/2}$',color="red")
        fig, ax, lines, labels  = plot_xy(x_val2, y_val, x_error=x_err2,err_amp=err_amp,fig=fig,ax=ax,lines=lines,labels=labels,y_label='',x_label=r'Coverage per configuration $\Phi_{1/2}$',color="black")
        
    X = np.linspace(0,1,1000)
    X[0] = 1e-9
    X[-1] = 1 - 1e-9
    line, = ax.plot(X,temperature_Phi_ising(X), color="orange")
    lines.append(line)
    labels.append("Ising binodal")
    labels[0] = "Lying molecules \n" + "Simulation data with $SEM$"
    labels[1] = "Standing molecules \n" + "Simulation data with $SEM$"
    ax.legend(lines, labels,loc="best")
    if not titel is None:
        ax.set_title(titel, fontsize=16)
    if filename is None:
        fig.savefig("Phi_12-T-diagramm")
    else:
        fig.savefig(filename)
#phaseDiagramm_Phi12_T(phi_mean,T_mean,cov_mean)

In [ ]:
def phaseDiagramm_T_n1(meanPhi1,T,coverage, xdim,err_amp=1, T_c=T_cIsing, filename=None, ax = None, fig=None, ylim=None, titel=None, y_label=True):
    #sim_original x xdim
    #sim_SEM
    #sim_SEM
    n1 = (np.sum(meanPhi1,1) / xdim)/coverage
    assert np.shape(n1) == np.shape(T)
    
    n_sim = np.shape(T)[0]
    listT = np.sort(list(set(T)))
    listCov = np.sort(list(set(coverage)))
    
    assert np.shape(listCov)[0] == 1#for now till I think about how to include multiple coverages
    if np.shape(listCov)[0] == 1:
        n1_mean = list()
        for t in listT:
            n = np.array([n1[sim] for sim in range(n_sim) if (t == T[sim])])
            n = mean_SEM(n)
            n1_mean.append(n)
        n1_mean = np.array(n1_mean)
        
        x_val = listT
        y_val = np.array(n1_mean[:,0])
        y_err = np.array(n1_mean[:,1])
        #Plot
        if ax is None:
            fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,7))
        if y_label:
            fig, ax, lines, labels  = plot_xy(x_val, y_val, y_error=y_err,err_amp=err_amp,fig=fig,ax=ax,x_label=r'Temperature $T$',y_label='Global fraction of lying\n'+r'molecules $x_1=\frac{N_1}{N}$', color="red")
        else:
            fig, ax, lines, labels  = plot_xy(x_val, y_val, y_error=y_err,err_amp=err_amp,fig=fig,ax=ax,x_label=r'Temperature $T$',y_label='', color="red")
        line = ax.axvline(x=T_c, color = "black")
        lines.append(line)
        labels.append(r'$T_c$')
        if not ylim is None:
            ax.set_ylim(ylim)
        ax.legend(lines,labels,loc="best")
        if not titel is None:
            ax.set_title(titel, fontsize=16)
        if filename is None:
            fig.savefig("Phi_12-T-diagramm")
        else:
            fig.savefig(filename)

#phaseDiagramm_T_n1(meanPhi1,T,coverage, xdim)

#single set of data
#xData = np.array(np.loadtxt("xFractionsFinal0.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal1.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal2.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal3.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal4.csv", dtype=float, delimiter=";", skiprows=1))
xData = np.array(np.loadtxt("xFractionsFinal5.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal6.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal7.csv", dtype=float, delimiter=";", skiprows=1))
#xData = np.array(np.loadtxt("xFractionsFinal8.csv", dtype=float, delimiter=";", skiprows=1))

#xData = np.array([xData]) #für einzelversuche
#[[xdim; T ; coverage ; meanPhi1[xdim]; meanPhi2[xdim]]]
xdim = int(xData[0,0])#assuming having always the same lattice length
n_sim = np.shape(xData)[0]
T = xData[:,1]
coverage = xData[:,2]
meanPhi1 = xData[:,3:xdim+3]
meanPhi2 = xData[:,xdim+3:]
#assert np.size(meanPhi2) == np.size(meanPhi1)
#meanPhi12 = np.array([meanPhi1, meanPhi2])

paras = fitMultipleDenstyTanh(xData, "density_fit_plot")
#fitMultipleDenstyTanhJustStanding(xData, "density_fit_plot_standing")
PhiFromParas = paras_to_Phi(xdim, paras)#i x sim x phase
PhiPhysical = makePhiPhysical(PhiFromParas)#i x sim x phase
phi_mean, T_mean, cov_mean = meanPhiPerT(PhiPhysical,T,coverage)#i x sim x phase x [phi,err]

In [ ]:
#phaseDiagramm_Phi12_T(phi_mean,T_mean,cov_mean,err_amp=2)
#phaseDiagramm_x_Phi_T(phi_mean,T_mean,cov_mean,err_amp=2)
#phaseDiagramm_x_T(phi_mean,T_mean,cov_mean,err_amp=2)
#phaseDiagramm_T_phi(phi_mean,T_mean,cov_mean,err_amp=2)
#phaseDiagramm_T_n1(meanPhi1,T,coverage, xdim,err_amp=2)

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,5), gridspec_kw={'width_ratios': [6, 7]})
#phaseDiagramm_Phi12_T(phi_mean,T_mean,cov_mean,filename="Final8_Phi12_xPhiT",ax=ax[0],fig=fig)
#phaseDiagramm_x_Phi_T(phi_mean,T_mean,cov_mean,filename="Final8_Phi12_xPhiT",ax=ax[1],fig=fig)

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,5))
#phaseDiagramm_T_phi(phi_mean,T_mean,cov_mean,err_amp=10,ax=ax,fig=fig, filename="Ising_T_Phi")

In [ ]:
#first set of data
xData = np.array(np.loadtxt("xFractionsFinal1.csv", dtype=float, delimiter=";", skiprows=1))

#[[xdim; T ; coverage ; meanPhi1[xdim]; meanPhi2[xdim]]]
xdim = int(xData[0,0])#assuming having always the same lattice length
n_sim = np.shape(xData)[0]
T_I = xData[:,1]
coverage_I = xData[:,2]
meanPhi1_I = xData[:,3:xdim+3]
meanPhi2_I = xData[:,xdim+3:]

paras_I = fitMultipleDenstyTanh(xData, "density_fit_plot")
PhiFromParas_I = paras_to_Phi(xdim, paras_I)#i x sim x phase
PhiPhysical_I = makePhiPhysical(PhiFromParas_I)#i x sim x phase
phi_mean_I, T_mean_I, cov_mean_I = meanPhiPerT(PhiPhysical_I,T_I,coverage_I)#i x sim x phase x [phi,err]

#second set of data
xData = np.array(np.loadtxt("xFractionsFinal6.csv", dtype=float, delimiter=";", skiprows=1))

#[[xdim; T ; coverage ; meanPhi1[xdim]; meanPhi2[xdim]]]
xdim = int(xData[0,0])#assuming having always the same lattice length
n_sim = np.shape(xData)[0]
T_II = xData[:,1]
coverage_II = xData[:,2]
meanPhi1_II = xData[:,3:xdim+3]
meanPhi2_II = xData[:,xdim+3:]

paras_II = fitMultipleDenstyTanh(xData, "density_fit_plot")
PhiFromParas_II = paras_to_Phi(xdim, paras_II)#i x sim x phase
PhiPhysical_II = makePhiPhysical(PhiFromParas_II)#i x sim x phase
phi_mean_II, T_mean_II, cov_mean_II = meanPhiPerT(PhiPhysical_II,T_II,coverage_II)#i x sim x phase x [phi,err]

#third set of data
xData = np.array(np.loadtxt("xFractionsFinal1.csv", dtype=float, delimiter=";", skiprows=1))

#[[xdim; T ; coverage ; meanPhi1[xdim]; meanPhi2[xdim]]]
xdim = int(xData[0,0])#assuming having always the same lattice length
n_sim = np.shape(xData)[0]
T_III = xData[:,1]
coverage_III = xData[:,2]
meanPhi1_III = xData[:,3:xdim+3]
meanPhi2_III = xData[:,xdim+3:]

paras_III = fitMultipleDenstyTanh(xData, "density_fit_plot")
PhiFromParas_III = paras_to_Phi(xdim, paras_III)#i x sim x phase
PhiPhysical_III = makePhiPhysical(PhiFromParas_III)#i x sim x phase
phi_mean_III, T_mean_III, cov_mean_III = meanPhiPerT(PhiPhysical_III,T_III,coverage_III)#i x sim x phase x [phi,err]

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,5))
#phaseDiagramm_Phi12_T(phi_mean_I,T_mean_I,cov_mean_I,ax=ax[0],fig=fig,filename="Phi_comparison_Phi12_T",titel='Global coverage $\Phi=0.5$')
#phaseDiagramm_Phi12_T(phi_mean_II,T_mean_II,cov_mean_II,ax=ax[1],fig=fig,filename="Phi_comparison_Phi12_T",titel='Global coverage $\Phi=0.7$',y_label=False)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,5))
phaseDiagramm_Phi12_T(phi_mean_I,T_mean_I,cov_mean_I,ax=ax[0],fig=fig,filename="rTF_comparison_Phi12_T",titel=r'$r_{Try Flip}=0.1$')
phaseDiagramm_Phi12_T(phi_mean_II,T_mean_II,cov_mean_II,ax=ax[1],fig=fig,filename="rTF_comparison_Phi12_T",titel='r$r_{Try Flip}=0.5$',y_label=False)

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,5))
#phaseDiagramm_T_n1(meanPhi1_I,T_I,coverage_I,xdim,err_amp=10,ax=ax[0],fig=fig,filename="Phi_comparison_T_x1",titel='Global coverage $\Phi=0.5$',ylim=(0,0.35))
#phaseDiagramm_T_n1(meanPhi1_II,T_II,coverage_II,xdim,err_amp=10,ax=ax[1],fig=fig,filename="Phi_comparison_T_x1",titel='Global coverage $\Phi=0.7$',ylim=(0,0.35), y_label=False)

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize = (12,5))
#phaseDiagramm_T_n1(meanPhi1_I,T_I,coverage_I,xdim,ax=ax[0],fig=fig,filename="h_comparison_T_x1",titel='$s = 0$',ylim=(0,0.35))
#phaseDiagramm_T_n1(meanPhi1_II,T_II,coverage_II,xdim,ax=ax[1],fig=fig,filename="h_comparison_T_x1",titel='$s = 0.4$',ylim=(0,0.35), y_label=False)
#phaseDiagramm_T_n1(meanPhi1_III,T_III,coverage_III,xdim,ax=ax[2],fig=fig,filename="h_comparison_T_x1",titel='$s = 12(T-T_c)$',ylim=(0,0.35), y_label=False)

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,5))
#phaseDiagramm_T_n1(meanPhi1_I,T_I,coverage_I,xdim,err_amp=10,ax=ax[0],fig=fig,filename="epsilon_comparison_T_x1",titel='$\epsilon=0.6$',ylim=(0,0.35))
#phaseDiagramm_T_n1(meanPhi1_II,T_II,coverage_II,xdim,err_amp=10,ax=ax[1],fig=fig,filename="epsilon_comparison_T_x1",titel='$\epsilon=0.35$',ylim=(0,0.35), y_label=False)

In [ ]:
#fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize = (12,5))
#phaseDiagramm_T_n1(meanPhi1_I,T_I,coverage_I,xdim,ax=ax[0],fig=fig,filename="alpha_comparison_T_x1",titel=r'$\alpha = \frac{3}{2T_c} \approx 2.644$',ylim=(0,0.35))
#phaseDiagramm_T_n1(meanPhi1_II,T_II,coverage_II,xdim,ax=ax[1],fig=fig,filename="alpha_comparison_T_x1",titel=r'$\alpha = 6$',ylim=(0,0.35), y_label=False)
#phaseDiagramm_T_n1(meanPhi1_III,T_III,coverage_III,xdim,ax=ax[2],fig=fig,filename="alpha_comparison_T_x1",titel=r'$\alpha = 12$',ylim=(0,0.35), y_label=False)

In [ ]:
def table_Phi_mean(T_mean,phi_mean):
    #i x sim x phase x [phi,err]
    rphi = np.round(phi_mean,4)
    n_sim = np.shape(T_mean)[0]
    for sim in range(n_sim):
        if T_mean[sim] < T_cIsing:
            print("${}$ & ${}\pm{}$ & ${}\pm{}$ & ${}\pm{}$ & ${}\pm{}$\\\\ \hline".format(T_mean[sim],
                                                                                      rphi[0,sim,0,0],
                                                                                      rphi[0,sim,0,1],
                                                                                      rphi[0,sim,1,0],
                                                                                      rphi[0,sim,1,1],
                                                                                      rphi[1,sim,0,0],
                                                                                      rphi[1,sim,0,1],
                                                                                      rphi[1,sim,1,0],
                                                                                      rphi[1,sim,1,1]))
#table_Phi_mean(T_mean,phi_mean)

def phaseDiagramm_T_n1_values(meanPhi1,T,coverage, xdim):
    #sim_original x xdim
    #sim_SEM
    #sim_SEM
    n1 = (np.sum(meanPhi1,1) / xdim)/coverage
    assert np.shape(n1) == np.shape(T)
    
    n_sim = np.shape(T)[0]
    listT = np.sort(list(set(T)))
    listCov = np.sort(list(set(coverage)))
    
    assert np.shape(listCov)[0] == 1#for now till I think about how to include multiple coverages
    if np.shape(listCov)[0] == 1:
        n1_mean = list()
        for t in listT:
            n = np.array([n1[sim] for sim in range(n_sim) if (t == T[sim])])
            n = mean_SEM(n)
            n1_mean.append(n)
        n1_mean = np.array(n1_mean)
        
        x_val = listT
        y_val = np.array(n1_mean[:,0])
        y_err = np.array(n1_mean[:,1])
        
        return x_val,y_val,y_err

    
T_liste = list()
x1_liste = list()
xe_liste = list()

for number in range(5,9):
    xData = np.array(np.loadtxt("xFractionsFinal{}.csv".format(number), dtype=float, delimiter=";", skiprows=1))
    #[[xdim; T ; coverage ; meanPhi1[xdim]; meanPhi2[xdim]]]
    xdim = int(xData[0,0])#assuming having always the same lattice length
    n_sim = np.shape(xData)[0]
    T = xData[:,1]
    coverage = xData[:,2]
    meanPhi1 = xData[:,3:xdim+3]
    meanPhi2 = xData[:,xdim+3:]

    x_val,y_val,y_err = phaseDiagramm_T_n1_values(meanPhi1,T,coverage, xdim)
    T_liste=x_val
    x1_liste.append(y_val)
    xe_liste.append(y_err)
    
x1_liste = np.array(x1_liste)
xe_liste = np.array(xe_liste)
x1_liste = np.round(x1_liste,4)
xe_liste = np.round(xe_liste,4)
for i in range(np.shape(T_liste)[0]):
    print("${}$ & ${}\pm{}$ & ${}\pm{}$ & ${}\pm{}$ & ${}\pm{}$\\\\ \hline".format(T_liste[i]
                                                                                   ,x1_liste[0,i]
                                                                                   ,xe_liste[0,i]
                                                                                   ,x1_liste[1,i]
                                                                                   ,xe_liste[1,i]
                                                                                   ,x1_liste[2,i]
                                                                                   ,xe_liste[2,i]
                                                                                   ,x1_liste[3,i]
                                                                                   ,xe_liste[3,i]))
                                                                                   
